In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.models import Model
from PIL import Image
import tensorflow as tf
import numpy as np
import imutils
import cv2


model = VGG16(weights="imagenet")


orig = cv2.imread("/content/sample_data/116.webp")
print(orig)
# resized = cv2.resize(orig, (224, 224))


image = load_img("/content/sample_data/116.webp", target_size=(224, 224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = imagenet_utils.preprocess_input(image)


preds = model.predict(image)
# print(preds)
i = np.argmax(preds[0])
# print(i)

decoded = imagenet_utils.decode_predictions(preds)
(imagenetID, label, prob) = decoded[0][0]
label = "{}: {:.2f}%".format(label, prob * 100)
print("[INFO] {}".format(label))

class GradCAM:
    def __init__(self, model, classIdx, layerName=None):

        self.model = model
        self.classIdx = classIdx
        self.layerName = layerName


        if self.layerName is None:
            self.layerName = self.find_target_layer()

    def find_target_layer(self):

        for layer in reversed(self.model.layers):
            if len(layer.output.shape) == 4:
                return layer.name


        raise ValueError("Could not find 4D layer. Cannot apply GradCAM.")

    def compute_heatmap(self, image, eps=1e-8):

        gradModel = Model(inputs=[self.model.inputs], outputs= [self.model.get_layer(self.layerName).output, self.model.output])

        with tf.GradientTape() as tape:

            inputs = tf.cast(image, tf.float32)
            (convOutputs, predictions) = gradModel(inputs)
            loss = predictions[:, self.classIdx]

        grads = tape.gradient(loss, convOutputs)

        castConvOutputs = tf.cast(convOutputs > 0, "float32")
        castGrads = tf.cast(grads > 0, "float32")
        guidedGrads = castConvOutputs * castGrads * grads


        convOutputs = convOutputs[0]
        guidedGrads = guidedGrads[0]


        weights = tf.reduce_mean(guidedGrads, axis=(0, 1))
        cam = tf.reduce_sum(tf.multiply(weights, convOutputs), axis=-1)


        (w, h) = (image.shape[2], image.shape[1])
        heatmap = cv2.resize(cam.numpy(), (w, h))


        numer = heatmap - np.min(heatmap)
        denom = (heatmap.max() - heatmap.min()) + eps
        heatmap = numer / denom
        heatmap = (heatmap * 255).astype("uint8")

        return heatmap

    def overlay_heatmap(self, heatmap, image, alpha=0.5,
        colormap=cv2.COLORMAP_JET):
        heatmap = cv2.applyColorMap(heatmap, colormap)
        output = cv2.addWeighted(image, alpha, heatmap, 1 - alpha, 0)


        return (heatmap, output)

cam = GradCAM(model, i)
heatmap = cam.compute_heatmap(image)


heatmap = cv2.resize(heatmap, (orig.shape[1], orig.shape[0]))
(heatmap, output) = cam.overlay_heatmap(heatmap, orig, alpha=0.5)

from google.colab.patches import cv2_imshow
output = cv2.resize(output, (orig.shape[1], output.shape[0]))


output = np.vstack([orig, heatmap, output])
output = imutils.resize(output, height=700)
cv2_imshow(output)
cv2.waitKey(0)
cv2.destroyAllWindows()



